In [3]:
import json

In [4]:
version="step-v2"

In [7]:
# Pipeline with Scaledown
stages_id = ["scaledown", "crop", "mirrorx", "mirrory", "bw", "rotate", "watermark"]
stages_suffix = ["resized", "cropped", "mirrorx", "mirrory", "bw", "rot", "watermarked"]

# Pipeline without Scaledown
stages_id = ["crop", "mirrorx", "mirrory", "bw", "rotate", "watermark"]
stages_suffix = ["cropped", "mirrorx", "mirrory", "bw", "rot", "watermarked"]

# Version 3 with scaledown in between
stages_id = ["crop", "scaledown", "mirror", "mirror_bw", "mirror_2rotate", "mirror_3rotate"]
stages_suffix = ["cropped", "resized", "mirror", "mirror_bw", "mirror_rot_rot", "mirror_rot_rot_rot_watermarked"]
moviepy_op_count = [1, 1, 1, 2, 3, 5]

In [8]:
filename = "ElephantsDream"
for stage, suffix in zip(stages_id, stages_suffix):
    input = {}

    input["lambdaARN"] = "arn:aws:lambda:us-west-2:075165449331:function:pipeline-{}-{}".format(stage, version)
    input["powerValues"] = [1024,2048,4096,8192]
    input["num"] = 5
    input["parallelInvocation"] = "true"
    input["strategy"] = "cost"

    input["payload"] = {}
    input["payload"]["filename"] = filename

    filename = filename + "_" + suffix

    print(json.dumps(input))

    with open('./../conf/{}.json'.format(stage), 'w') as f:
        json.dump(input, f)

    


{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-crop-step-v2", "powerValues": [1024, 2048, 4096, 8192], "num": 5, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-scaledown-step-v2", "powerValues": [1024, 2048, 4096, 8192], "num": 5, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-mirror-step-v2", "powerValues": [1024, 2048, 4096, 8192], "num": 5, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped_resized"}}
{"lambdaARN": "arn:aws:lambda:us-west-2:075165449331:function:pipeline-mirror_bw-step-v2", "powerValues": [1024, 2048, 4096, 8192], "num": 5, "parallelInvocation": "true", "strategy": "cost", "payload": {"filename": "ElephantsDream_cropped_resized_mirror"}}
{"lambdaARN": "arn:aw